In [21]:
from data import find_board, split_boxes, get_perspective
import cv2
import numpy as np

import easyocr
reader = easyocr.Reader(['en'])
import imutils

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [19]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def contrast(img):
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    return enhanced_img

def blued(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    return thresh

In [56]:
# Read image
img = cv2.imread('test3.jpg')
saturated = increase_brightness(img)
blue = blued(saturated)

cv2.imwrite("sat.png", saturated)
cv2.imwrite("blue.png", blue)
input_size = 48

contours, hierarchy = cv2.findContours(blue.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# contours  = imutils.grab_contours(keypoints)

newimg = cv2.drawContours(saturated.copy(), contours, -1, (0, 255, 0), 3)
cv2.imwrite("Contour.png", newimg)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:15]
location = None
print(len(contours))
# Finds rectangular contour
for i,contour in enumerate(contours):
    epsilon = 0.1*cv2.arcLength(contour,True)
    newimg = cv2.drawContours(img.copy(), contour, -1, (0, 255, 0), 3)
    cv2.imwrite("Contour"+str(i)+".png", newimg)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    print(len(approx))
    if len(approx) == 4:
        location = approx
        break
location = np.roll(location, -np.argmin(location.sum(axis=1)), axis=0)
result = get_perspective(img, location)

15
4


In [55]:
np.roll(location, -np.argmin(location.sum(axis=1)), axis=0)

array([[[ 265,  367]],

       [[ 307, 2764]],

       [[2767, 2765]],

       [[2720,  240]]], dtype=int32)

In [53]:
np.argmin(location.sum(axis=1)), location.shape, location.reshape()

(1,
 (4, 1, 2),
 array([[[2720,  240]],
 
        [[ 265,  367]],
 
        [[ 307, 2764]],
 
        [[2767, 2765]]], dtype=int32))

In [63]:
def split_boxes2(board):
    """Takes a sudoku board and split it into 81 cells. 
        each cell contains an element of that board either given or an empty cell."""
    rows = np.vsplit(board,9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r,9)
        for box in cols:
            # box = cv2.resize(box, (input_size, input_size))/255.0
            # cv2.imshow("Splitted block", box)
            # cv2.waitKey(50)
            boxes.append(box)
    return boxes

In [64]:
gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

cv2.imwrite("gray.png", gray)
# print(gray.shape)
rois = split_boxes2(gray)
# rois = np.array(rois).reshape(-1, input_size, input_size, 1)

In [65]:

cv2.imwrite("r.png", rois[0])

True

In [ ]:

# extract board from input image
board, location = find_board(blue)

cv2.imwrite("board.png", board)
# print(gray.shape)
rois = split_boxes(gray)
# rois = np.array(rois).reshape(-1, 9, 9, 1)

In [66]:
m = []

for i,r in enumerate(rois):
    predicted = reader.readtext(r, allowlist='0123456789')
    m.append(int(predicted[0][-2]) if predicted and len(predicted) > 0 else 0)

In [67]:
np.array(m).reshape((9,9))

array([[0, 3, 0, 0, 9, 0, 2, 0, 1],
       [0, 0, 0, 0, 1, 0, 4, 7, 0],
       [0, 0, 0, 7, 0, 6, 0, 9, 0],
       [0, 0, 0, 0, 0, 8, 3, 0, 6],
       [0, 2, 0, 0, 0, 0, 0, 5, 0],
       [4, 0, 5, 3, 0, 0, 0, 0, 0],
       [0, 1, 0, 6, 0, 2, 0, 0, 0],
       [0, 6, 2, 0, 4, 0, 0, 0, 0],
       [3, 0, 9, 0, 8, 0, 0, 2, 0]])